In [1]:
import pycountry as py
import pandas as pd
import re
import numpy as np
import nltk 
from nltk import PorterStemmer, download
from nltk.corpus import stopwords
from string import punctuation
download('stopwords')
import collections as c

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/biglifa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_collection=pd.read_csv('../data/df_collection.csv')
df_article=pd.read_csv('../data/df_article.csv')
df_auteur=pd.read_csv('../data/df_auteur.csv')
df_redige=pd.read_csv('../data/df_redige.csv')

# Collection

In [3]:
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180.0,4523.0,1987.0,2008.0,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320.0,6776.0,1950.0,2002.0,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441.0,12292.0,1894.0,2015.0,philosophie,Peeters
3,1957-7966,Population,320.0,12794.0,1946.0,2004.0,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49.0,814.0,1985.0,2001.0,démographie,MSH Poitiers


In [4]:
df_collection[df_collection['issn'].isnull()]

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Supprimer des collections non referencées
df_collection.drop([26,207,243], inplace=True)

In [6]:
df_collection=df_collection.fillna('')

In [7]:
#Convertir en entier certaines colonnes
df_collection['nb_numeros']=df_collection['nb_numeros'].astype(int)
df_collection['nb_document']=df_collection['nb_document'].astype(int)
df_collection['pre_année_pub']=df_collection['pre_année_pub'].astype(int)
df_collection['der_année_pub']=df_collection['der_année_pub'].astype(int)
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180,4523,1987,2008,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320,6776,1950,2002,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441,12292,1894,2015,philosophie,Peeters
3,1957-7966,Population,320,12794,1946,2004,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49,814,1985,2001,démographie,MSH Poitiers


In [8]:
def verifie_doublons(df):
    """
        Cette fonction verifie s'il y a des doublons sur une colonne donnée en paramètre
    
    """
    list_=df.values.tolist()
    c_=c.Counter()
    for m in list_:
        c_[m]+=1
    m_=c_.most_common()
    lm=[]
    for m in m_:
        if m[1]<=1:
            break
        else:
            lm.append(m[0])
    return lm

In [9]:
verifie_doublons(df_collection['issn'])

[]

In [11]:
df_collection.to_csv(sep=',', path_or_buf='../data/t_collection.csv',index=False)

# Article


In [12]:
df_article[['num_doc','annee_publication']]=df_article[['num_doc','annee_publication']].astype(int)
df_article.head()

,num_doc,title,type,annee_publication,num_publication,abstract,issn
0,1,La fuite des cerveaux : le cas de l'ex-URSS,article,1992,1155,La fuite des cerveaux : le cas de l'ex-URSS......,2262-3353
1,2,"Croissance, redistribution et inégalités dans ...",article,1999,50-3,"Croissance, redistribution et inégalités dans ...",1950-6694
2,3,Que nous devons-nous les uns aux autres ? Le c...,article,2008,106-4,La fuite des cerveaux est-elle un phénomène in...,1783-1768
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...,compte-rendu,1970,25-5,Chorafas Dimitri N. — La fuite des cerveaux. L...,1957-7966
4,5,Au-delà de la « fuite des cerveaux » : la mobi...,article,1997,13-1,Au-delà de la « fuite des cerveaux » : la mobi...,1777-5418


In [14]:
df_article[df_article['num_doc'].isnull()]

,num_doc,title,type,annee_publication,num_publication,abstract,issn


In [15]:
df_article=df_article.fillna('')

In [16]:
t_article=df_article[['num_doc','title','annee_publication','issn']]
abstract=df_article[['num_doc','abstract']]

In [17]:
verifie_doublons(t_article['num_doc'])

[]

In [18]:
t_article.to_csv(sep=',', path_or_buf='../data/t_article.csv',index=False)

# Mot article

In [19]:
abstract.head()

,num_doc,abstract
0,1,La fuite des cerveaux : le cas de l'ex-URSS......
1,2,"Croissance, redistribution et inégalités dans ..."
2,3,La fuite des cerveaux est-elle un phénomène in...
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...
4,5,Au-delà de la « fuite des cerveaux » : la mobi...


In [20]:
def nettoyer_abstract(df):
    """
        Cette fonction nettoie les abstracts. Enleves les stopwords, les ponctuations et ne prend que les radicales des mots
    
    """
       
    #Recuperer les stopswords 
    sr = nltk.corpus.stopwords.words('french')
    sc = ['cet','cette','celui','celle','celles','leurs','quoi','quel','lequel','laquelle','lesquels','lesquelles']
    stopwordss=list(set().union(sr,sc))
  
    
    #Recuperer les ponctuations
    poncts=punctuation
    poncts+='’'
    
    #lemmatisation
    stemmer=nltk.stem.snowball.FrenchStemmer()
    
    for i in df.index:
        doc=df.loc[i,['abstract']][0]
        clean_tokens=[]
        doc=str(doc)
        
        #Enlever les ponctuations
        for p in poncts :
            doc=doc.replace(p," ")
            
        doc=doc.lower()
        tokens=doc.split()

        #Enlever les stopwords et raciniser
        clean_tokens=[stemmer.stem(token) for token in tokens if (token not in stopwordss and len(token)>3)]

        #Enlever les chiffres
        clean_tokenss=[token for token in clean_tokens if token.isnumeric()==False]
        
        df.loc[i,['abstract']]=' '.join(clean_tokenss)
    
    return df

In [21]:
abstract=nettoyer_abstract(abstract)
abstract.head()

/home/biglifa/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,num_doc,abstract
0,1,fuit cerveau urss exod inte¬ rieur cess accéle...
1,2,croissanc redistribu inégal model fuit cerveau...
2,3,fuit cerveau phénomen individuel légitim socia...
3,4,choraf dimitr fuit cerveau conquêt sav choraf ...
4,5,delà fuit cerveau mobil person haut qualifi ou...


In [22]:
#Recuper l'ensemble du corpus
corpus=(' '.join(abstract['abstract'].values.tolist())).split()
corpus[:5]

['fuit', 'cerveau', 'urss', 'exod', 'inte¬']

In [23]:
#Compter chaque mot du coprus
count_mot=c.Counter()
for mot in corpus:
    count_mot[mot]+=1
len(count_mot)

39633

In [24]:
#Je prends les 10 mots les plus frequents comme mots clé
mots_cles=count_mot.most_common(12)
mots_cles

[('europ', 26814),
 ('fuit', 4275),
 ('des', 3823),
 ('plus', 3661),
 ('pay', 3105),
 ('cerveau', 2753),
 ('polit', 2382),
 ('franc', 2161),
 ('européen', 2101),
 ('tout', 2066),
 ('comm', 2058),
 ('entre', 1999)]

In [25]:
mots_cles=[mot[0] for mot in mots_cles]
mots_cles.remove('cerveau')
mots_cles.remove('fuit')
mots_cles

['europ',
 'des',
 'plus',
 'pay',
 'polit',
 'franc',
 'européen',
 'tout',
 'comm',
 'entre']

In [26]:
#Creer le dataframe article_comporte_mot
t_mot_article=pd.DataFrame(columns=['mot','num_article','frequence'])
j=0
for i in abstract.index:
    for mot in mots_cles:
        text=abstract.loc[i,['abstract']][0].split()
        if mot in text:
            freq=text.count(mot)
            t_mot_article.loc[j,:]=[mot,i+1,freq]
            j+=1
t_mot_article.head()

,mot,num_article,frequence
0,plus,2,1
1,polit,3,1
2,europ,4,1
3,plus,4,1
4,pay,4,1


In [27]:
t_mot_article.to_csv(sep=',', path_or_buf='../data/t_mot_article.csv',index=False)

# Pays article

In [28]:
#Une fonction qui renvoie la liste des pays du monde
def f_pays():
    list_pays=[]
    for pays in list(py.countries):
        a= re.search("name='"+'.+'+", numeric=",str(pays))
        if a:
            a=a.group(0)[6:-11]
            list_pays.append(a.lower())
    return list_pays

In [29]:
#Creer le dataframe pays_article

t_pays_article=pd.DataFrame(columns=['nom_pays','num_article'])
list_pays=f_pays()
j=0
for i in abstract.index:
    text=abstract.loc[i,['abstract']][0]
    for pays in list_pays:
        p=re.search(pays,text)
        if p:
            t_pays_article.loc[j,:]=[p.group(0),i+1]
            j+=1
t_pays_article.head()

,nom_pays,num_article
0,réunion,15
1,poland,36
2,oman,43
3,pakistan,62
4,egypt,66


In [30]:
t_pays_article.to_csv(sep=',', path_or_buf='../data/t_pays_article.csv',index=False)

# Auteur

In [31]:
df_auteur.head()

,num,nom,annee_naissance,annee_decès,bio
0,414356,Kortunov Andrei,NaN,NaN,NaN
1,140232,Bouyssou Rachel,NaN,NaN,NaN
2,24789,Docquier Frédéric,NaN,NaN,Docteur en sciences économiques. - Macroéconom...
3,61222,Rapoport Hillel,1963.0,NaN,NaN
4,731465,Chauvier Stéphane,1959.0,NaN,Maître de conférences de philosophie à l'Unive...


In [33]:
df_auteur[df_auteur['num'].isnull()]

,num,nom,annee_naissance,annee_decès,bio


In [34]:
#Remplacer les Nan par vide
df_auteur=df_auteur.fillna('')
df_auteur.head()

,num,nom,annee_naissance,annee_decès,bio
0,414356,Kortunov Andrei,,,
1,140232,Bouyssou Rachel,,,
2,24789,Docquier Frédéric,,,Docteur en sciences économiques. - Macroéconom...
3,61222,Rapoport Hillel,1963,,
4,731465,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...


In [40]:
bios=df_auteur[['num','bio']]

In [35]:
#Ajouter les professions des auteurs
df_auteur['profession']=''
for i in df_auteur.index:
    bio=df_auteur.loc[i,['bio']][0]
    if bio!='':
        b=bio[:3]
        prof=re.search(r""+b+"\w*(['\(\) ]\w*)*[,\.-]",bio)
        if prof:
            df_auteur.loc[i,'profession']=prof.group(0)[:-1]
df_auteur.head()

,num,nom,annee_naissance,annee_decès,bio,profession
0,414356,Kortunov Andrei,,,,
1,140232,Bouyssou Rachel,,,,
2,24789,Docquier Frédéric,,,Docteur en sciences économiques. - Macroéconom...,Docteur en sciences économiques
3,61222,Rapoport Hillel,1963,,,
4,731465,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...,Maître de conférences de philosophie à l'Unive...


In [36]:
t_auteur=df_auteur[['num','nom','annee_naissance','annee_decès','profession']]
t_auteur.head()

,num,nom,annee_naissance,annee_decès,profession
0,414356,Kortunov Andrei,,,
1,140232,Bouyssou Rachel,,,
2,24789,Docquier Frédéric,,,Docteur en sciences économiques
3,61222,Rapoport Hillel,1963,,
4,731465,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...


In [37]:
verifie_doublons(t_auteur['num'])

[]

In [38]:
t_auteur.to_csv(sep=',', path_or_buf='../data/t_auteur.csv',index=False)

# Pays auteur


In [41]:
list_pays=f_pays()
nationalite=pd.DataFrame(columns=['num','nom_pays'])
j=0
for i in bios.index:
    bio=bios.loc[i,['bio']][0]
    num=bios.loc[i,['num']][0]
    if bio!='':
        bio=bio.lower()
        for pays in list_pays:
            pp=re.search(pays,bio)
            if pp:
                nationalite.loc[j,:]=[num,pp.group(0)]
                j+=1
nationalite.head()

,num,nom_pays
0,33007,france
1,259994,france
2,249315,france
3,215901,france
4,234171,france


In [44]:
nationalite.to_csv(sep=',', path_or_buf='../data/t_nationalite.csv',index=False)

# Redige

In [45]:
df_redige.head()

,nom,num_doc,role
0,414356,1.0,NaN
1,140232,1.0,Traducteur
2,24789,2.0,NaN
3,61222,2.0,NaN
4,731465,3.0,NaN


In [46]:
df_redige['num_doc']=df_redige['num_doc'].astype(int)

In [47]:
t_redige=df_redige[['nom','num_doc']]
t_redige.head()

,nom,num_doc
0,414356,1
1,140232,1
2,24789,2
3,61222,2
4,731465,3


In [48]:
t_redige.rename(columns={'nom':'num'}, inplace=True)

/home/biglifa/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [49]:
t_redige.head()

,num,num_doc
0,414356,1
1,140232,1
2,24789,2
3,61222,2
4,731465,3


In [50]:
t_redige.to_csv(sep=',', path_or_buf='../data/t_redige.csv',index=False)